In [73]:
import jsonlines
import json
import polars as pl

In [74]:
# lines contains all log lines
lines = []
with jsonlines.open('logs.jsonl') as reader:
    for log in reader:
        lines.append(json.JSONDecoder().decode(log['line']))
lines[:3]

[{'level': 30,
  'time': 1748016000227,
  'pid': 1,
  'clientAddress': '172.22.0.1',
  'method': 'POST',
  'url': 'https://enotaryo.crabdance.com/otp?/approve',
  'body': {'txnId': '2406642583', 'otp': '111111'},
  'msg': 'transaction patch attempt'},
 {'level': 30,
  'time': 1748016000227,
  'pid': 1,
  'clientAddress': '172.22.0.1',
  'method': 'POST',
  'url': 'https://enotaryo.crabdance.com/otp?/approve',
  'requestHandleTime': 0.03625600039958954},
 {'level': 30,
  'time': 1748016000577,
  'pid': 1,
  'clientAddress': '172.22.0.1',
  'method': 'PATCH',
  'url': 'https://enotaryo.crabdance.com/api/otpTransaction',
  'requestHandleTime': 0.016787000000476837}]

In [75]:
# lines containing all testing log lines
testing_lines = []
with jsonlines.open('tester_logs.jsonl') as reader:
    for log in reader:
        testing_lines.append(json.JSONDecoder().decode(log['line']))
testing_lines[:3]

[{'level': 30,
  'time': 1748016000849,
  'pid': 1020205,
  'start': '11:59:45 PM',
  'document': 0,
  'msg': 'finished test #a-0'},
 {'level': 30,
  'time': 1748016002122,
  'pid': 1020205,
  'start': '11:59:45 PM',
  'msg': 'navigating to document: a-1'},
 {'level': 30,
  'time': 1748016002141,
  'pid': 1020205,
  'start': '11:59:45 PM',
  'msg': 'clicking file input: a-1'}]

In [76]:
# routines contains all lines relevant to routines
routines = list(filter(lambda x: x.get('routine', None) is not None, lines))

routines[:3]

[{'level': 30,
  'time': 1748016000928,
  'pid': 1,
  'clientAddress': '172.22.0.1',
  'method': 'PATCH',
  'url': 'https://enotaryo.crabdance.com/api/otpTransaction',
  'routine': 'b3',
  'elapsedTime': 350.1578550003469,
  'msg': 'routine b3'},
 {'level': 30,
  'time': 1748016003717,
  'pid': 1,
  'clientAddress': '172.22.0.1',
  'method': 'POST',
  'url': 'https://enotaryo.crabdance.com/api/document',
  'routine': 'a3.1',
  'elapsedTime': 369.9873789995909,
  'msg': 'routine a3.1'},
 {'level': 30,
  'time': 1748016004130,
  'pid': 1,
  'clientAddress': '172.22.0.1',
  'method': 'PATCH',
  'url': 'https://enotaryo.crabdance.com/api/otpTransaction',
  'routine': 'b4',
  'elapsedTime': 2852.8324120007455,
  'msg': 'routine b4'}]

In [77]:
len(routines)

104757

In [78]:
# put all routine times into a dictionary
from collections import defaultdict

routine_times = defaultdict(list)

for routine in routines:
    routine_name = routine['routine']
    elapsed_time = routine['elapsedTime']
    routine_times[routine_name].append(elapsed_time)

routine_times = dict(routine_times)
routine_mean_times = dict(routine_times)

sorted(list(routine_times.keys()))

['a1',
 'a2',
 'a3.1',
 'a3.2',
 'a4',
 'a5',
 'a6',
 'a7',
 'a8',
 'a9',
 'b1',
 'b2',
 'b3',
 'b4',
 'b5.1',
 'b5.2',
 'c1',
 'c2',
 'c3',
 'c4',
 'c5',
 'd1']

In [79]:
# calculate average time for each routine
for routine_key in routine_times:
    routine_mean_times[routine_key] = sum(routine_times[routine_key]) / len(routine_times[routine_key])

routine_mean_times = dict(sorted(routine_times.items(), key=lambda item: item[1], reverse=True))
routine_mean_times

{'b1': [3903.437633998692,
  2657.6919940002263,
  3499.522257000208,
  3149.181913997978,
  3858.0870549976826,
  2440.8372940011322,
  2533.7731360010803,
  2985.7050740011036,
  3016.4484449997544,
  1643.2008369974792,
  2679.5264849998057,
  2539.1617750003934,
  2555.9831429980695,
  2377.281130000949,
  3261.3966230005026,
  2382.988630000502,
  2722.934637002647,
  2709.450913000852,
  2503.7867319993675,
  1569.2893779985607,
  2704.4622489996254,
  2552.774469997734,
  2565.639797002077,
  2546.828832998872,
  2610.4832499995828,
  2342.174389000982,
  2528.3040260002017,
  1568.2743769995868,
  2497.911628998816,
  2648.3917860016227,
  2743.7525090016425,
  2728.77797299996,
  2341.363582998514,
  2607.7853090018034,
  2400.9937859997153,
  2528.1033909991384,
  2518.6806429997087,
  2713.078803997487,
  2801.0008090026677,
  2559.359379000962,
  2643.892035998404,
  2690.314649000764,
  2698.990199998021,
  2996.469388999045,
  2864.5711230002344,
  2759.642772000283,
  26

In [80]:
# calculate the success rate for the document upload routine

# find all document upload routines
import re

document_upload_routine_codes = sorted(list(filter(lambda x: re.match("a.*", x), routine_times.keys())))
document_upload_routine_codes

['a1', 'a2', 'a3.1', 'a3.2', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9']

In [81]:
prev = ""
document_upload_ratios = dict()

for code in document_upload_routine_codes:
    if prev:
        print(f"{prev} to {code}: {len(routine_times[code])} / {len(routine_times[prev])}")
        document_upload_ratios[f"{prev} to {code}"] = len(routine_times[code]) / len(routine_times[prev])
    prev = code
    
document_upload_ratios

a1 to a2: 7006 / 7000
a2 to a3.1: 7008 / 7006
a3.1 to a3.2: 7006 / 7008
a3.2 to a4: 7002 / 7006
a4 to a5: 6998 / 7002
a5 to a6: 6994 / 6998
a6 to a7: 6989 / 6994
a7 to a8: 6990 / 6989
a8 to a9: 6989 / 6990


{'a1 to a2': 1.000857142857143,
 'a2 to a3.1': 1.0002854695974879,
 'a3.1 to a3.2': 0.9997146118721462,
 'a3.2 to a4': 0.9994290608050242,
 'a4 to a5': 0.9994287346472437,
 'a5 to a6': 0.9994284081166047,
 'a6 to a7': 0.9992851015155848,
 'a7 to a8': 1.000143081985978,
 'a8 to a9': 0.999856938483548}

In [82]:
document_approval_routine_codes = sorted(list(filter(lambda x: re.match("b.*", x), routine_times.keys())))
document_approval_routine_codes

['b1', 'b2', 'b3', 'b4', 'b5.1', 'b5.2']

In [83]:
prev = ""
document_approval_ratios = dict()

for code in document_approval_routine_codes:
    if prev:
        print(f"{prev} to {code}: {len(routine_times[code])} / {len(routine_times[prev])}")
        document_approval_ratios[f"{prev} to {code}"] = len(routine_times[code]) / len(routine_times[prev])
    prev = code
    
document_approval_ratios

b1 to b2: 3248 / 3248
b2 to b3: 3250 / 3248
b3 to b4: 3248 / 3250
b4 to b5.1: 2912 / 3248
b5.1 to b5.2: 2911 / 2912


{'b1 to b2': 1.0,
 'b2 to b3': 1.000615763546798,
 'b3 to b4': 0.9993846153846154,
 'b4 to b5.1': 0.896551724137931,
 'b5.1 to b5.2': 0.9996565934065934}

In [84]:
document_denial_routine_codes = sorted(list(filter(lambda x: re.match("c.*", x), routine_times.keys())))
document_denial_routine_codes

['c1', 'c2', 'c3', 'c4', 'c5']

In [85]:
prev = ""
document_denial_ratios = dict()

for code in document_denial_routine_codes:
    if prev:
        print(f"{prev} to {code}: {len(routine_times[code])} / {len(routine_times[prev])}")
        document_denial_ratios[f"{prev} to {code}"] = len(routine_times[code]) / len(routine_times[prev])
    prev = code
    
document_denial_ratios

c1 to c2: 3193 / 3190
c2 to c3: 3193 / 3193
c3 to c4: 3190 / 3193
c4 to c5: 3191 / 3190


{'c1 to c2': 1.0009404388714733,
 'c2 to c3': 1.0,
 'c3 to c4': 0.9990604447228312,
 'c4 to c5': 1.0003134796238244}